In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

In [ ]:
schema = "gans_local"
host = "localhost"
user = "root"
password = mySQL_pass
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
def tomorrows_flight_arrivals(icao_list):
    #icao_list = ["EDDB", "EDDH", "EDDM"]
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    list_for_arrivals_df = []

    for icao in icao_list:

        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": API_key,
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_resp = response.json()

            arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
            arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
            arrivals_df["arrival_airport_icao"] = icao
            arrivals_df["data_retrieved_on"] = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
            arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "arrival_terminal", "departure_city", "departure_airport_icao", "data_retrieved_on"]]

            # fixing arrival_time
            arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str.split("+").str[0]
            #arrivals_df.columns = [col.capitalize() for col in arrivals_df.columns]
            list_for_arrivals_df.append(arrivals_df)
        
    if list_for_arrivals_df:
       final_df = pd.concat(list_for_arrivals_df, ignore_index=True)
       final_df.to_sql('flight_arrival',con=connection_string,if_exists='append',index=False)
    return final_df

In [ ]:
icao_list = ["EDDB", "EDDH", "EDDM"]
tomorrows_flight_arrivals (icao_list)